In [ ]:
import plotly.figure_factory as ff
import pandas as pd
from datetime import datetime, timedelta
import re # Импортируем библиотеку для работы с регулярными выражениями

# --- Упрощенная Структура ИСР с примерными длительностями и зависимостями ---
# Формат: (ID, Короткий текст работы, Уровень в иерархии, Длительность в днях, Зависимость от ID (Finish-to-Start))
# Зависимости здесь упрощенные и последовательные для примера
wbs_items_timed = [
    ('1', 'Инициация проекта', 1, 0, None), # Фазы как вехи или суммарные задачи (0 длительность)
    ('1.1', 'Анализ предметной области', 2, 10, '1'), # Зависит от начала Фазы 1
    ('1.1.1', 'Анализ текущих диагностических процессов', 3, 5, '1.1'), # Зависит от 1.1
    ('1.1.3', 'Определение актуальности и потребностей', 3, 5, '1.1.1'), # Зависит от 1.1.1
    ('1.2', 'Определение целей и требований', 2, 8, '1.1.3'), # Зависит от 1.1.3
    ('1.2.1', 'Формулирование бизнес-целей', 3, 3, '1.2'), # Зависит от 1.2
    ('1.2.2', 'Сбор и анализ функциональных требований', 3, 5, '1.2.1'), # Зависит от 1.2.1
    ('1.5', 'Составление ТЗ / Устава проекта', 2, 5, '1.2.2'), # Зависит от 1.2.2

    ('2', 'Планирование проекта', 1, 0, '1.5'), # Фаза 2 начинается после завершения 1.5 (последняя задача Фазы 1)
    ('2.1', 'Детализация и описание работ', 2, 3, '2'), # Зависит от начала Фазы 2
    ('2.4', 'Определение ресурсов', 2, 5, '2.1'), # Зависит от 2.1
    ('2.4.1', 'Потребность в команде', 3, 3, '2.4'), # Зависит от 2.4
    ('2.4.4', 'Формирование бюджета', 3, 2, '2.4.1'), # Зависит от 2.4.1
    ('2.5', 'Разработка графика (Диаграмма Ганта)', 2, 7, '2.4.4'), # Зависит от 2.4.4
    ('2.6', 'Анализ рисков и стратегии реагирования', 2, 10, '2.5'), # Зависит от 2.5
    ('2.10', 'Разработка плана управления', 2, 5, '2.6'), # Зависит от 2.6

    ('3', 'Реализация проекта', 1, 0, '2.10'), # Фаза 3 начинается после завершения 2.10 (последняя задача Фазы 2)
    ('3.1', 'Создание инфраструктуры', 2, 40, '3'), # Зависит от начала Фазы 3
    ('3.1.1', 'Требования к оборудованию', 3, 5, '3.1'), # Зависит от 3.1
    ('3.1.4', 'Настройка ПО для ИИ', 3, 10, '3.1.1'), # Зависит от 3.1.1
    ('3.1.5', 'Развертывание ИИ-модели', 3, 15, '3.1.4'), # Зависит от 3.1.4
    ('3.2', 'Разработка модулей системы', 2, 70, '3'), # Параллельно с инфраструктурой, зависит от начала Фазы 3
    ('3.2.1', 'Разработка модуля данных', 3, 20, '3.2'), # Зависит от начала 3.2
    ('3.2.4', 'Разработка Frontend', 3, 40, '3.2.1'), # Зависит от 3.2.1
    ('3.3', 'Разработка ИИ-компонентов', 2, 50, '3.2.4'), # Может начаться после частичной готовности Frontend
    ('3.3.1', 'Выбор ИИ-модели (T5-large)', 3, 5, '3.3'), # Зависит от начала 3.3
    ('3.3.4', 'Разработка промптов', 3, 15, '3.3.1'), # Зависит от 3.3.1
    ('3.3.5', 'Настройка вывода', 3, 10, '3.3.4'), # Зависит от 3.3.4

    ('4', 'Оценка, тест-завершение проекта', 1, 0, '3.3.5'), # Фаза 4 начинается после завершения 3.3.5 (последняя задача реализации)
    ('4.1', 'Тестирование решения', 2, 20, '4'), # Зависит от начала Фазы 4
    ('4.1.1', 'Модульное тестирование', 3, 5, '4.1'), # Зависит от 4.1
    ('4.1.3', 'Нагрузочное тестирование', 3, 7, '4.1.1'), # Зависит от 4.1.1
    ('4.1.5', 'Клиническая валидация', 3, 10, '4.1.3'), # Зависит от 4.1.3
    ('4.2', 'Оценка реализуемости и эффективности', 2, 10, '4.1.5'), # Зависит от завершения тестирования
    ('4.2.1', 'Анализ соответствия ТЗ', 3, 5, '4.2'), # Зависит от начала 4.2
    ('4.2.3', 'Экономическая оценка', 3, 5, '4.2.1'), # Зависит от 4.2.1
    ('4.4', 'Развертывание системы', 2, 5, '4.2.3'), # Зависит от завершения оценки
    ('4.7', 'Завершение и отчетность', 2, 3, '4.4'), # Зависит от развертывания
]

# --- Расчет дат на основе длительности и зависимостей ---
# Словарь для хранения дат завершения задач по их ID
task_finish_dates = {}
# Начальная дата проекта
project_start_date = datetime(2025, 1, 1)

# Список данных для Plotly
gantt_data = []

# Сортируем задачи по ID. Это важно для корректного расчета зависимостей,
# т.к. мы обрабатываем их в порядке возрастания ID.
wbs_items_timed.sort(key=lambda item: item[0])

for item_id, item_text, item_level, duration, dependency_id in wbs_items_timed:
    start_date = project_start_date

    # Определяем дату старта на основе зависимости
    if dependency_id:
        if dependency_id in task_finish_dates:
            # Старт на следующий день после завершения зависимости
            start_date = task_finish_dates[dependency_id] + timedelta(days=1)
        else:
            # Это должно быть исключено, если зависимости логически корректны и WBS отсортирован.
            # Если это происходит, значит, зависимость на задачу, которая еще не была обработана
            # или не существует. Для надежности можно поднять ошибку или оставить предупреждение.
            print(f"Warning: Dependency ID '{dependency_id}' not found or processed yet for task '{item_id}'. Setting start to project start date.")
            start_date = project_start_date
    elif item_id == '1': # Специально для первой задачи, если у нее нет зависимости
        start_date = project_start_date
    else: # Для других задач верхнего уровня без явной зависимости, но входящих в иерархию
        # Если это задача верхнего уровня (например, "2", "3", "4"), и она не '1',
        # то ее старт должен быть привязан к завершению последней задачи предыдущей фазы.
        # Это уже учтено в wbs_items_timed, но на случай ошибок в данных,
        # убедимся, что у таких задач есть dependency_id.
        pass

    finish_date = start_date + timedelta(days=duration)

    # Сохраняем дату завершения текущей задачи в словаре
    task_finish_dates[item_id] = finish_date

    # Добавляем задачу в список данных для Plotly
    gantt_data.append(dict(
        Task=f'{item_id} - {item_text}', # Метка задачи на графике
        Start=start_date.strftime('%Y-%m-%d'),
        Finish=finish_date.strftime('%Y-%m-%d'),
        Resource=f'Фаза {item_id.split(".")[0]}' # Используем номер фазы как ресурс для группировки/раскраски
    ))

# --- Создание Диаграммы Ганта с Plotly ---

fig = ff.create_gantt(gantt_data,
                      index_col='Resource', # Группировать/раскрашивать по фазам
                      show_colorbar=True,
                      group_tasks=True, # Группировать задачи по ресурсу (фазе) на оси Y
                      title='Диаграмма Ганта проекта: Система анализа неструктурированных медицинских данных',
                      bar_width=0.6,
                      # show_arrows=False, # Эту строку нужно удалить, так как она вызывает ошибку
                     )

# --- Настройка внешнего вида ---
fig.update_layout(
    hovermode='closest', 
    xaxis=dict(
        tickfont=dict(size=12), 
        tickformat='%Y-%m',  # Изменено на формат год-месяц
        dtick='M1',          # Отображать метки каждого месяца
        showgrid=True,       # Отображение сетки
        gridwidth=1,         # Ширина линий сетки
        gridcolor='LightGrey'# Цвет линий сетки
    ),
    yaxis=dict(
        tickfont=dict(size=12), 
        categoryorder='category ascending' 
    ),
)
"""
fig.update_layout(
    hovermode='closest', # Режим отображения всплывающих подсказок
    xaxis=dict(
        tickfont=dict(size=10), # Размер шрифта меток оси X
        tickformat='%Y-%m-%d',  # Формат меток даты: год-месяц-день
        dtick='D1',             # Интервал меток: 1 день. Используйте 'W1' для еженедельных меток.
        showgrid=True,          # Отображение сетки
        gridwidth=1,            # Ширина линий сетки
        gridcolor='LightGrey'   # Цвет линий сетки
    ),
    yaxis=dict(
        tickfont=dict(size=10), # Размер шрифта меток оси Y
        categoryorder='category ascending' # Порядок отображения задач (сверху вниз в порядке списка)
    ),
)
"""
# --- Отображение диаграммы ---
fig.show()

# --- Сохранение диаграммы (опционально) ---
# Для сохранения в файл (нужны библиотеки kaleido: pip install -U kaleido)
#fig.write_image("gantt_medical_ai.png") # Сохранить как PNG
#fig.write_html("gantt_medical_ai.html") # Сохранить как интерактивный HTML файл